In [1]:
import numpy as np
import pandas as pd 
import cv2
import os
import tqdm
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import shuffle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class OralCancerDataset(Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""

    def __init__(self, path_to_images, path_to_csv = None):
        
        # Passing the path to the train csv file reads the data from the csv with the labels
        # If None is passes insted only the images in the image folder is loaded (wich is useful for the test set)
        
        self.path_to_images = path_to_images
        self.path_to_csv = path_to_csv
        self.transform= transforms.Compose([
    transforms.Normalize(                      
            mean=[0.485, 0.456, 0.406],                
            std=[0.229, 0.224, 0.225]                  
            ),
    transforms.ToTensor()
])
        
        if self.path_to_csv is not None:
            self.df = pd.read_csv(self.path_to_csv)
    
    def __len__(self):
        if self.path_to_csv:
            return len(self.df)
        else:
            return len(glob.glob(self.path_to_images + '/*.jpg'))
    
    def __getitem__(self, idx):
        
        if self.path_to_csv:
            data = self.df.iloc[idx]
            image = cv2.imread(os.path.join(self.path_to_images, data['Name']), -1)
            label = data['Diagnosis']
            
            # You can input torchvision (or other) transforms and directly augment the data
            # if self.transform:
            #image = self.transform(image)
            # ..
            # image = transforms.functional.to_pil_image(image)
            # image = transforms.functional.adjust_contrast(image, 2)
            image = transforms.functional.to_tensor(image)
            image = transforms.functional.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            
            return image, label
            
        else:
            name = 'image_' + str(idx) + '.jpg'
            image = cv2.imread(os.path.join(self.path_to_images, name), -1)

            # image = transforms.functional.to_pil_image(image)
            # image = transforms.functional.adjust_contrast(image, 2)
            image = transforms.functional.to_tensor(image)
            image = transforms.functional.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            
            return image, name

In [16]:
class VGG_4Cancer_Classification(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG_4Cancer_Classification, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer4 = nn.Sequential(

            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            #nn.Dropout(0.2),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer7 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
            )
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        self.layer9 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            #nn.Dropout(0.5),
            #nn.Linear(7*7*512, 4096),
            nn.Linear(2*2*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            #nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(

            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        #print(out.shape)
        out = out.reshape(out.size(0), -1)
        #print(out.shape)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [17]:
path_to_csv = 'Data/train.csv'
path_to_train_images = path_to_val_images = 'Data/train'
path_to_test_images = 'Data/test'

df = pd.read_csv(path_to_csv)
df['PatID'] = df['Name'].str[:7]

val_df = df[df['Name'].str.contains("pat_025|pat_096|pat_081")][['Name','Diagnosis']].copy()

path_to_valcsv = 'Input/val_label.csv'
val_df.to_csv(path_to_valcsv,index=False)

train_df = df[~df['Name'].str.contains("pat_025|pat_096|pat_081")][['Name','Diagnosis']].copy()

path_to_traincsv = 'Input/train_label.csv'
train_df.to_csv(path_to_traincsv,index=False)


In [18]:
train_dataset = OralCancerDataset(path_to_train_images, path_to_traincsv)

val_dataset = OralCancerDataset(path_to_val_images, path_to_valcsv)

test_dataset = OralCancerDataset(path_to_test_images, path_to_csv = None)

train_dataloader = DataLoader(train_dataset,
batch_size=32,
shuffle=True,
num_workers=0) 

val_dataloader= DataLoader(val_dataset,
batch_size=32,
shuffle=True,
num_workers=0) 


test_dataloader = DataLoader(test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0)

In [19]:
#len(train_dataloader)*32 +
len(val_dataloader)*32 

20160

In [20]:
#HYPERPARAMS

num_classes = 2
num_epochs = 20
batch_size = 32
learning_rate = 0.005
l1_lambda = 0.001
weight_decay_=0.005 #L2 Loss

model = VGG_4Cancer_Classification(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = weight_decay_, momentum = 0.90)  



In [21]:
# import torchsummary
# torchsummary.summary(model,(3,128,128))

In [22]:
#TRAINING

total_step = len(train_dataloader)+len(val_dataloader)
df_train={'Loss':[], 'Accuracy':[]}
df_val={'Loss':[], 'Accuracy':[]}
for epoch in range(num_epochs):
    print ('Epoch [{}/{}]' 
                   .format(epoch+1, num_epochs))
    i=0
    correct = 0
    total = 0
    loss_train=[]
    for (images, labels) in tqdm.tqdm(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        images=images.permute(0,1,2,3)
        # Forward pass
        outputs = model(images.float())
        loss = criterion(outputs, labels)
        loss_train.append(loss.item())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #L1 loss
       
        # l1_norm = sum(abs(p).sum()
        #           for p in model.parameters())

        # loss = loss + l1_lambda * l1_norm

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i=i+1

    df_train['Loss'].append(np.mean(loss_train))
    df_train['Accuracy'].append(100 * correct / total)
    print ('Epoch [{}/{}], Step [{}/{}], Accuracy: {:0.4f}; Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step,100 * correct / total, np.mean(loss_train)))
    
    loss_val=[]
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            images=images.permute(0,1,2,3)
            outputs = model(images.float())
            loss_v = criterion(outputs, labels)
            loss_val.append(loss_v.item())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        df_val['Loss'].append(np.mean(loss_val))
        df_val['Accuracy'].append(100 * correct / total)
        print('Accuracy of the network on the {} validation images: {:.4f} % ; Loss - {:.4f}'.format(len(val_dataloader)*32, 100 * correct / total,np.mean(loss_val))) 


Epoch [1/20]


100%|██████████| 1666/1666 [01:55<00:00, 14.46it/s]


Epoch [1/20], Step [1667/2296], Accuracy: 72.9946; Loss: 1.0017
Accuracy of the network on the 20160 validation images: 59.5223 % ; Loss - 0.6930
Epoch [2/20]


100%|██████████| 1666/1666 [01:55<00:00, 14.45it/s]


Epoch [2/20], Step [1667/2296], Accuracy: 78.4411; Loss: 0.4667
Accuracy of the network on the 20160 validation images: 54.1391 % ; Loss - 0.8404
Epoch [3/20]


...

Epoch [18/20], Step [1667/2296], Accuracy: 95.0208; Loss: 0.1288
Accuracy of the network on the 20160 validation images: 66.5293 % ; Loss - 1.4017
Epoch [19/20]


100%|██████████| 1666/1666 [01:54<00:00, 14.57it/s]


Epoch [19/20], Step [1667/2296], Accuracy: 92.3445; Loss: 0.1940
Accuracy of the network on the 20160 validation images: 67.4331 % ; Loss - 1.2530
Epoch [20/20]


100%|██████████| 1666/1666 [01:54<00:00, 14.59it/s]


Epoch [20/20], Step [1667/2296], Accuracy: 95.2273; Loss: 0.1236
Accuracy of the network on the 20160 validation images: 58.8072 % ; Loss - 1.7241


In [23]:
#SAVING EVAL DATA FOR INFERENCE
train_summary = pd.DataFrame(df_train)

tr_filename='Inference/Trial_8/'
if not os.path.exists(tr_filename):
    os.mkdir(tr_filename)
train_summary.to_csv(tr_filename+'train.csv', index = False)
print("File {} saved successfully!".format(tr_filename+'train.csv'))

val_summary = pd.DataFrame(df_val)
val_filename='Inference/Trial_8/'
val_summary.to_csv(val_filename+'val.csv', index = False)
print("File {} saved successfully!".format(val_filename+'val.csv'))

File Inference/Trial_8/train.csv saved successfully!
File Inference/Trial_8/val.csv saved successfully!


In [24]:
# Test Prediction
df={'Name':[], 'Diagnosis':[]}
i=0
with torch.no_grad():
    correct = 0
    total = 0
    for images, names in test_dataloader:
        images = images.to(device)

        images=images.permute(0,1,2,3)
        outputs = model(images.float()).cpu()
        _, predicted = torch.max(outputs.data, 1)

        df['Name'].extend(names)
        df['Diagnosis'].extend(predicted.detach().numpy())
        i+=1
        print('Test images {}/{} done.'.format(i*32,len(test_dataloader)*32),end='\r') 

test_preds = pd.DataFrame(df)
test_filename='Predictions/test_preds_8.csv'

test_preds.to_csv(test_filename, index = False)
print("File {} saved successfully!".format(test_filename))


File Predictions/test_preds_8.csv saved successfully!


In [25]:
#SAVING MODEL DATA 
model_path="Model/final_model_8.pt"
torch.save(model,model_path)
print("Model saved successfully!\nPath: {}".format(model_path))

Model saved successfully!
Path: Model/final_model_8.pt
